In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute, quantum_info

# TODO
* Ensure initial random vector is the same for all optimizers, just to be fair
* Tests
* Write docs and references
* Optimize optimizer parameters, eta's, tolerances, etc.
* Run it once with 1000 iterations, and go for lunch, and hope jupyter doesn't die
* Benchmark execution time for each optimizer
* Find parameters with dwave and compare, perhaps?
* Add a noise model to the simulator and reevaluate results

In [2]:
L = 10
N_QUBITS = 4
CURRENT_LAYER = 0
PHI = quantum_info.random_statevector(2**N_QUBITS)
ITERATIONS = 100 # Iterations for each step in each opimizer. Start with 50 to test. Then increase gradually and go for a (long) coffee

backend = Aer.get_backend("qasm_simulator")

In [3]:
# Unitary 4 Qubit gate with Rz's full entanglement
def even_gate(theta, index, qubits):
    name = "Ue"+str(index)+"("+str(theta)+")"
    u_even = QuantumCircuit(4, name=name)
    u_even.rz(theta,qubits)

    for t in qubits:
        i = t
        for z in range(len(qubits)-t-1):
            u_even.cz(t, i + 1)
            i += 1

    return u_even

# Unitary 4 Qubit gate with Rx's and no entanglement
def odd_gate(theta, index, qubits):
    name = "Uo"+str(index)+"("+str(theta)+")"
    u_odd = QuantumCircuit(4, name=name)
    u_odd.rx(theta, qubits)

    return u_odd

In [4]:
def calculate_random_theta():
    return np.random.uniform(0, 2*np.pi)

In [5]:
def get_random_thetas(layers):
    thetas = []
    for i in range(layers):
        thetas.append(calculate_random_theta())
    return thetas

In [6]:
def get_variational_circuit(thetas):
    layers = CURRENT_LAYER
    n_qubits = N_QUBITS
    
    qubits = [i for i in range(n_qubits)]
    circuit = QuantumCircuit(n_qubits)
    theta = 0
    for i in range(layers):
        circuit.append(even_gate(thetas[theta], i, qubits), qubits)
        theta += 1
        circuit.append(odd_gate(thetas[theta], i, qubits), qubits)
        theta += 1
        
    return circuit

def run_circuit(thetas):
    layers = CURRENT_LAYER
    n_qubits = N_QUBITS
    circuit = get_variational_circuit(thetas)
    
    simulator = Aer.get_backend("statevector_simulator")
    job = execute(circuit, backend=simulator, shots=1024)
    result = job.result()
    
    return result.get_statevector(circuit)


In [7]:
def objective_function(thetas):
    psi_theta = run_circuit(thetas)
    cost = np.linalg.norm(psi_theta - PHI.data)
    return cost

In [8]:
# Test Circuit
thetas = get_random_thetas(L*2)
CURRENT_LAYER = 3
circuit = get_variational_circuit(thetas)
circuit.draw()

┌────────────────────────┐┌─────────────────────────┐»
q_0: ┤0                       ├┤0                        ├»
     │                        ││                         │»
q_1: ┤1                       ├┤1                        ├»
     │  Ue0(4.73366734262213) ││  Uo0(5.508857905216674) │»
q_2: ┤2                       ├┤2                        ├»
     │                        ││                         │»
q_3: ┤3                       ├┤3                        ├»
     └────────────────────────┘└─────────────────────────┘»
«     ┌──────────────────────────┐┌──────────────────────────┐»
«q_0: ┤0                         ├┤0                         ├»
«     │                          ││                          │»
«q_1: ┤1                         ├┤1                         ├»
«     │  Ue1(1.9501610946458077) ││  Uo1(0.3876145748966889) │»
«q_2: ┤2                         ├┤2                         ├»
«     │                          ││                          │»
«q_3: ┤3                         ├┤3                         ├»
«     └──────────────────────────┘└──────────────────────────┘»
«     ┌──────────────────────────┐┌──────────────────────────┐
«q_0: ┤0                         ├┤0                         ├
«     │                          ││                          │
«q_1: ┤1                         ├┤1                         ├
«     │  Ue2(1.7960588931498565) ││  Uo2(4.6197440006599075) │
«q_2: ┤2                         ├┤2                         ├
«     │                          ││                          │
«q_3: ┤3                         ├┤3                         ├
«     └──────────────────────────┘└──────────────────────────┘

In [9]:
random_results = []

for layers in list(range(1, L+1)):
    CURRENT_LAYER = layers
    optimization_results = []    
    
    for i in range(ITERATIONS):
        thetas = get_random_thetas(layers * 2)
        psi_theta = run_circuit(thetas)
        optimization_results.append(np.linalg.norm(psi_theta - PHI.data))
    
    distance = min(optimization_results)

    print("Layers: ",layers, "| Distance: ",distance)
    random_results.append(distance)

Layer:  1 | Distance:  1.1828208370652735
Layer:  2 | Distance:  1.0341537818106317
Layer:  3 | Distance:  1.14747166108551
Layer:  4 | Distance:  1.072194186256165
Layer:  5 | Distance:  1.007292462431876
Layer:  6 | Distance:  1.1299987490037748
Layer:  7 | Distance:  1.0651139827901885
Layer:  8 | Distance:  1.0666270222354108
Layer:  9 | Distance:  1.144093159113435
Layer:  10 | Distance:  1.0729608462327271


In [10]:
## Trial with COBYLA
from qiskit.aqua.components.optimizers import COBYLA

cobyla_results = []
optimizer = COBYLA(maxiter=ITERATIONS, tol=0.0001, rhobeg=0.3)

for layers in list(range(1, L+1)):
    CURRENT_LAYER = layers
    params = get_random_thetas(layers * 2)
    bounds = []
    for t in params: bounds.append((0, 2*np.pi))

    ret = optimizer.optimize(num_vars=layers*2, 
                             objective_function=objective_function, 
                             initial_point=params,
                             variable_bounds=bounds)
    psi_theta = run_circuit(ret[0])
    print("Layers: ",layers,"| Distance: ",ret[1])
    cobyla_results.append(ret[1])

Layer:  1 | Distance:  1.1828159539517846
Layer:  2 | Distance:  0.9714782811942295
Layer:  3 | Distance:  0.9621956716563813
Layer:  4 | Distance:  1.005780079078617
Layer:  5 | Distance:  0.9868988583701669
Layer:  6 | Distance:  0.952991538082414
Layer:  7 | Distance:  0.9245738696911062
Layer:  8 | Distance:  0.9716895697046967
Layer:  9 | Distance:  1.0106979643808656
Layer:  10 | Distance:  0.9206973181378116


In [12]:
## Trial with SPSA
from qiskit.aqua.components.optimizers import SPSA

spsa_results = []
optimizer = SPSA(maxiter=ITERATIONS, c0=0.03, c1=0.01)

for layers in list(range(1, L+1)):
    CURRENT_LAYER = layers
    params = get_random_thetas(layers * 2)
    bounds = []
    for t in params: bounds.append((0, 2*np.pi))
        
    ret = optimizer.optimize(num_vars=layers*2, 
                             objective_function=objective_function, 
                             initial_point=params,
                             variable_bounds=bounds)
    psi_theta = run_circuit(ret[0])
    print("Layers: ",layers, "| Distance: ",ret[1])
    spsa_results.append(ret[1])

Layer:  1 | Distance:  1.3297084569225137
Layer:  2 | Distance:  1.2198557558798833
Layer:  3 | Distance:  1.0457045308256425
Layer:  4 | Distance:  0.9293058475430477
Layer:  5 | Distance:  1.3008947135556803
Layer:  6 | Distance:  1.4788826048815855
Layer:  7 | Distance:  1.4626639282133242
Layer:  8 | Distance:  1.4716090519651948
Layer:  9 | Distance:  1.1137059630017239
Layer:  10 | Distance:  1.6215125205890302


In [ ]:
## Trial with AQGD
from qiskit.aqua.components.optimizers import AQGD

aqgd_results = []
optimizer = AQGD(maxiter=ITERATIONS, tol=0.0001, eta=0.01)

for layers in list(range(1, L+1)):
    CURRENT_LAYER = layers
    params = get_random_thetas(layers * 2)
    bounds = []
    for t in params: bounds.append((0, 2*np.pi))
        
    ret = optimizer.optimize(num_vars=layers*2, 
                             objective_function=objective_function, 
                             initial_point=params,
                             variable_bounds=bounds)
    psi_theta = run_circuit(ret[0])
    print("Layers: ",layers, "| Distance: ",ret[1])
    aqgd_results.append(ret[1])

Layer:  1 | Distance:  1.4289045732471282
Layer:  2 | Distance:  1.2167656903819957
Layer:  3 | Distance:  1.2310673243000565
Layer:  4 | Distance:  1.3558845657045704
Layer:  5 | Distance:  1.4177809107365003
Layer:  6 | Distance:  1.3441781971912092
Layer:  7 | Distance:  1.2579303592430437
Layer:  8 | Distance:  1.5831021553065103
Layer:  9 | Distance:  1.3299912555135858


In [ ]:
# Evaluating all optimizers

x_axis = list(range(1, L+1))
plt.xticks(x_axis)
plt.plot(x_axis, cobyla_results, label="COBYLA")
plt.plot(x_axis, spsa_results, label="SPSA")
plt.plot(x_axis, aqgd_results, label="AQGD")
plt.plot(x_axis, random_results, label="Random Thetas")
plt.title('Cost')
plt.legend()
plt.xlabel('Layers')
plt.ylabel('Distance')